In [1]:
from google.colab import files  # If you're using Colab
uploaded = files.upload()

Saving harrier.pdf to harrier.pdf
Saving Nexon.pdf to Nexon.pdf
Saving Safari.pdf to Safari.pdf
Saving Tigor.pdf to Tigor.pdf


In [3]:
import pdfplumber
import os

pdf_files = ["Nexon.pdf", "Safari.pdf", "Tigor.pdf", "harrier.pdf"]

def extract_text_from_pdf(file_path):
    all_text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                all_text += page_text + "\n"
    return all_text

all_docs = []

for file in pdf_files:
    full_text = extract_text_from_pdf(file)
    all_docs.append({"file": file, "text": full_text})

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=32
)

split_docs = []

for doc in all_docs:
    chunks = splitter.create_documents([doc["text"]])
    for chunk in chunks:
        chunk.metadata = {"source": doc["file"]}
    split_docs.extend(chunks)

In [8]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 46.6 MB/s eta 0:00:00


In [9]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedder = SentenceTransformer("all-MiniLM-L6-v2")
texts = [doc.page_content for doc in split_docs]
embeddings = embedder.encode(texts, convert_to_tensor=True)

dimension = embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.cpu().numpy())

retrieval_data = [{"text": doc.page_content, "source": doc.metadata["source"]} for doc in split_docs]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
faiss.write_index(index, "/content/drive/MyDrive/DS Lab/tata_vector.index")

In [13]:
import json

with open("/content/drive/MyDrive/DS Lab/tata_metadata.json", "w") as f:
    json.dump(retrieval_data, f)